In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import mplfinance 
from renkodf import Renko
from scipy.signal import lfilter
# import MetaTrader5 as mt5

In [2]:
filepath = "C:/Users/WilliamFetzner/Documents/Trading/"
anytrading_filepath = "/gym-anytrading/gym_anytrading/datasets/data/"

In [2]:
# collect the actual data from csv file
# filename = "C:/Users/WilliamFetzner/Documents/Trading/DAT_XLSX_EURUSD_M1_2023.xlsx"

# exported from tickstory

yr_tickstory_csvs = []
for year in range(17, 22):
    yr_tickstory_csvs.append(f"EURUSD_11_15_{year}_to_11_15_{year+1}.csv")
tickstory_csvs = ["EURUSD_11_15_22_to_11_14_23.csv", "3mo_EURUSD.csv", "EURUSD_2_13_to_3_6.csv", 
                    "EURUSD_3_6_to_3_8_24.csv", "EURUSD_3_8_to_3_12_24.csv", "EURUSD_3_12_to_3_22_24.csv",
                    "EURUSD_3_23_to_3_30_24.csv"]
tickstory_csvs.extend(yr_tickstory_csvs)
tickstory_csvs


['EURUSD_11_15_22_to_11_14_23.csv',
 '3mo_EURUSD.csv',
 'EURUSD_2_13_to_3_6.csv',
 'EURUSD_3_6_to_3_8_24.csv',
 'EURUSD_3_8_to_3_12_24.csv',
 'EURUSD_3_12_to_3_22_24.csv',
 'EURUSD_3_23_to_3_30_24.csv',
 'EURUSD_11_15_17_to_11_15_18.csv',
 'EURUSD_11_15_18_to_11_15_19.csv',
 'EURUSD_11_15_19_to_11_15_20.csv',
 'EURUSD_11_15_20_to_11_15_21.csv',
 'EURUSD_11_15_21_to_11_15_22.csv']

In [3]:
last_to_add = ['EURUSD_11_15_20_to_11_15_21.csv',
 'EURUSD_11_15_21_to_11_15_22.csv']

In [1]:
# # df = pd.DataFrame()
# for df_name in last_to_add:
#     # check if filepath + "EURUSD_11_15_17_to_3_30_24.csv" exists
#     if not os.path.exists(filepath + "EURUSD_full_tickstory_data.csv"):
#         df = pd.DataFrame()
#     else:
#         df = pd.read_csv(filepath + "EURUSD_full_tickstory_data.csv")
#     df_new = pd.read_csv(filepath + df_name)
#     print(len(df_new), df_name)
#     df = pd.concat([df, df_new], ignore_index=True)
#     # save to csv
#     df.to_csv(filepath + "EURUSD_full_tickstory_data.csv", index=False)

In [6]:
def prep_data_fx(df):
    df = df.reset_index(drop=True)
    # drop any duplicate rows
    df = df.drop_duplicates()
    # convert date to datetime
    df['datetime'] = pd.to_datetime(df['Timestamp'], format='%Y%m%d %H:%M:%S:%f')
    # sort by datetime
    df = df.sort_values(by='datetime')
    # rename bid price to close
    df.rename(columns={'Bid price':'close'}, inplace=True)
    df_ready = df.set_index('datetime')
    # adjust the datetime 7 hrs ahead to match market time
    df_ready.index = df_ready.index + pd.Timedelta(hours=7)
    ohlc = df_ready[['close']]
    return ohlc

In [7]:
def generate_renko(df, brick_size, ending=''):
    # create a renko chart from the ohlc_dec_1 dataframe
    r_full = Renko(df, brick_size=brick_size)
    # create a new dataframe from the renko features
    renko_full_data = r_full.renko_df()
    # multiply brick_size by 10,000 to get a whole number
    brick_size_str = str(int(brick_size*100000))
    # save renko to csv
    renko_full_data.to_csv(f'{filepath}{anytrading_filepath}renko_full_data_{brick_size_str}{ending}.csv')
    return renko_full_data

In [4]:
# df = pd.read_csv(filepath + "EURUSD_full_tickstory_data.csv")

In [10]:
# df19_20 = pd.read_csv(filepath + "EURUSD_11_15_19_to_11_15_20.csv")
# df20_21 = pd.read_csv(filepath + "EURUSD_11_15_20_to_11_15_21.csv")

In [8]:
df.tail()

,Timestamp,Bid price,Ask price,Bid volume,Ask volume
126790936,20201115 18:59:51:975,1.18447,1.18451,0.75,1.50
126790937,20201115 18:59:52:076,1.18447,1.18450,0.75,0.75
126790938,20201115 18:59:52:178,1.18446,1.18451,1.12,4.50
126790939,20201115 18:59:56:232,1.18447,1.18452,0.75,13.86
126790940,20201115 18:59:58:934,1.18448,1.18450,0.75,0.75


In [ ]:
# # find out if there are any missing data from df20_21 that is not in df
# missing_data = df20_21.loc[~df20_21['Timestamp'].isin(df['Timestamp'])]
# missing_data

,Timestamp,Bid price,Ask price,Bid volume,Ask volume


In [9]:
df = prep_data_fx(df)
# df.to_csv(filepath + "EURUSD_full_tickstory_data_prepped.csv")

In [10]:
renko_full_data = generate_renko(df, 0.00081, ending='_initial')

In [11]:
renko_full_data = pd.read_csv(f"{filepath}{anytrading_filepath}renko_full_data_81_initial.csv")
renko_full_data.head()

,datetime,open,high,low,close,volume
0,2017-11-15 09:01:29.782,1.17936,1.18017,1.17846,1.18017,12545.0
1,2017-11-15 09:09:28.118,1.18017,1.18098,1.17990,1.18098,775.0
2,2017-11-15 10:01:52.936,1.18098,1.18179,1.18035,1.18179,4685.0
3,2017-11-15 10:08:27.340,1.18179,1.18260,1.18162,1.18260,793.0
4,2017-11-15 10:50:31.737,1.18260,1.18341,1.18121,1.18341,3910.0


In [14]:
renko_full_data = renko_full_data.loc[:, ['datetime', 'close']]
# get the last row of the renko full data
last_row = pd.DataFrame(renko_full_data.iloc[-1]).T
last_row

,datetime,close
21552,2024-03-29 15:40:11.502,1.07973


In [ ]:

df_new = pd.read_csv(f'{filepath}/EURUSD_11_15_20_to_11_15_21.csv')
ohlc = prep_data_fx(df_new)
# import the renko full data dataframe
renko_full_data = pd.read_csv(f"{filepath}{anytrading_filepath}renko_full_data_81_initial.csv")
renko_full_data = renko_full_data.loc[:, ['datetime', 'close']]
# get the last row of the renko full data
last_row = pd.DataFrame(renko_full_data.iloc[-1]).T
# insert that at the beginning of ohlc
ohlc = pd.concat([pd.DataFrame([last_row]), ohlc], ignore_index=True)
# find the year of the first date and the year of the last date in ohlc
year = ohlc.index[0].year
year_end = ohlc.index[-1].year
renko_full_data = generate_renko(ohlc, 0.00081, ending=f'_{year}_{year_end}')